## [NLP FROM SCRATCH: CLASSIFYING NAMES WITH A CHARACTER-LEVEL RNN](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html#nlp-from-scratch-classifying-names-with-a-character-level-rnn)

##### We will be building and training a basic character-level RNN to classify words. This tutorial, along with the following two, show how to do preprocess data for NLP modeling “from scratch”, in particular not using many of the convenience functions of torchtext, so you can see how preprocessing for NLP modeling works at a low level.

##### A character-level RNN reads words as a series of characters - outputting a prediction and “hidden state” at each step, feeding its previous hidden state into each next step. We take the final prediction to be the output, i.e. which class the word belongs to.

#### Specifically, we’ll train on a few thousand surnames from 18 languages of origin, and predict which language a name is from based on the spelling:

In [1]:
from glob import glob

In [2]:
import string

In [3]:
from tqdm import tqdm
import urllib
from zipfile import ZipFile
import os

In [4]:
url = "https://download.pytorch.org/tutorial/data.zip"

In [5]:
home = os.environ['HOME']
data_dir = f"{home}/torch/"
tar_file = data_dir + url.split('/')[-1]

In [6]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

In [7]:
if not os.path.isdir(data_dir):
    os.mkdir(data_dir)

In [8]:
with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=tar_file) as t:
    urllib.request.urlretrieve(url=url, filename=tar_file, reporthook=t.update_to)

/home/drclab/torch/data.zip: 2.88MB [00:01, 2.79MB/s]                            


In [9]:

with ZipFile(tar_file, "r") as zip:
    zip.extractall(data_dir)

In [10]:
for r, d, files in os.walk(data_dir):
    print(r, d, files)

/home/drclab/torch/ ['data'] ['data.zip']
/home/drclab/torch/data ['names'] ['eng-fra.txt']
/home/drclab/torch/data/names [] ['Arabic.txt', 'Irish.txt', 'Japanese.txt', 'Spanish.txt', 'Vietnamese.txt', 'Korean.txt', 'Portuguese.txt', 'Greek.txt', 'Polish.txt', 'Russian.txt', 'Dutch.txt', 'Scottish.txt', 'French.txt', 'German.txt', 'Chinese.txt', 'Czech.txt', 'Italian.txt', 'English.txt']


In [11]:
glob(data_dir+"data/names/*.txt")

['/home/drclab/torch/data/names/Arabic.txt',
 '/home/drclab/torch/data/names/Irish.txt',
 '/home/drclab/torch/data/names/Japanese.txt',
 '/home/drclab/torch/data/names/Spanish.txt',
 '/home/drclab/torch/data/names/Vietnamese.txt',
 '/home/drclab/torch/data/names/Korean.txt',
 '/home/drclab/torch/data/names/Portuguese.txt',
 '/home/drclab/torch/data/names/Greek.txt',
 '/home/drclab/torch/data/names/Polish.txt',
 '/home/drclab/torch/data/names/Russian.txt',
 '/home/drclab/torch/data/names/Dutch.txt',
 '/home/drclab/torch/data/names/Scottish.txt',
 '/home/drclab/torch/data/names/French.txt',
 '/home/drclab/torch/data/names/German.txt',
 '/home/drclab/torch/data/names/Chinese.txt',
 '/home/drclab/torch/data/names/Czech.txt',
 '/home/drclab/torch/data/names/Italian.txt',
 '/home/drclab/torch/data/names/English.txt']

In [12]:
all_letters = string.ascii_letters +" .,;'"

In [13]:
n_letters =  len(all_letters)

In [14]:
n_letters

57

In [15]:
import unicodedata

In [16]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

In [17]:
unicodeToAscii('Ślusàrski')

'Slusarski'

In [18]:
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [19]:
category_lines = {}
all_categories = []

In [20]:
def findFiles(path): return glob(path)

In [21]:
for filename in findFiles('/home/drclab/torch/data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

In [22]:
category_lines.keys()

dict_keys(['Arabic', 'Irish', 'Japanese', 'Spanish', 'Vietnamese', 'Korean', 'Portuguese', 'Greek', 'Polish', 'Russian', 'Dutch', 'Scottish', 'French', 'German', 'Chinese', 'Czech', 'Italian', 'English'])

In [23]:
n_categories = len(all_categories)

In [24]:
n_categories

18

_____

To represent a single letter, we use a “one-hot vector” of size <1 x n_letters>. A one-hot vector is filled with 0s except for a 1 at index of the current letter, e.g. "b" = <0 1 0 0 0 ...>.

To make a word we join a bunch of those into a 2D matrix <line_length x 1 x n_letters>.

In [25]:
all_letters.find('x')

23

In [26]:
def letterToIndex(letter):
    return all_letters.find(letter)

In [27]:
import torch

In [28]:
# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

In [29]:
letterToTensor('b')

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])

In [30]:
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [31]:
lineToTensor('John')

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0

![rnn](https://i.imgur.com/Z2xbySO.png)

In [32]:
import torch.nn as nn

In [41]:
class RNN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim) -> None:
        super(RNN, self).__init__()
        self.hidden_size = hidden_dim
        self.i2h = nn.Linear(in_dim+hidden_dim, hidden_dim)
        self.i2o = nn.Linear(in_dim+hidden_dim, out_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.concat((input, hidden), 1)
        hidden = self.i2h(combined)
        out = self.i2o(combined)
        out = self.softmax(out)
        return out, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [42]:
n_letters

57

In [54]:
n_categories

18

In [43]:
n_hidden = 128

In [44]:
input = letterToTensor('A')

In [52]:
input.shape

torch.Size([1, 57])

In [46]:
hidden =  torch.zeros(1, n_hidden)

In [51]:
hidden.shape

torch.Size([1, 128])

In [50]:
torch.concat((input, hidden), 1).shape

torch.Size([1, 185])

In [53]:
rnn = RNN(n_letters, n_hidden, n_categories)

In [56]:
out, next_hidden = rnn(input, hidden)

In [57]:
out

tensor([[-2.9904, -2.8022, -2.9068, -2.8616, -2.8730, -2.8390, -2.9207, -2.8630,
         -2.9119, -2.8643, -2.9019, -2.8509, -2.9244, -2.9277, -2.9152, -2.9770,
         -2.8846, -2.8321]], grad_fn=<LogSoftmaxBackward0>)

In [58]:
next_hidden

tensor([[-0.0390,  0.0139,  0.0473, -0.1212, -0.0720,  0.0009,  0.0137, -0.0030,
         -0.0161, -0.1334,  0.0171,  0.0030, -0.0155, -0.0094, -0.0073, -0.0357,
         -0.0206, -0.0050,  0.0040,  0.0236,  0.0351,  0.1133, -0.0243,  0.1265,
         -0.0379, -0.0553,  0.0457,  0.0020,  0.0300,  0.0900,  0.0932, -0.0154,
         -0.0703,  0.0048, -0.0254,  0.0041, -0.0408,  0.0635,  0.0383,  0.0632,
         -0.0892, -0.0642, -0.0350, -0.0108,  0.0075, -0.0578, -0.0427,  0.0906,
          0.0012,  0.0835,  0.0463, -0.0895, -0.0068, -0.0868,  0.0066,  0.0335,
         -0.0403, -0.0231,  0.0216, -0.0939, -0.0106, -0.0793,  0.0676, -0.0101,
         -0.0555,  0.0400,  0.0525, -0.0103, -0.0892, -0.0081, -0.0870, -0.0618,
         -0.0577, -0.0009, -0.0056,  0.0171, -0.1155,  0.0831,  0.0508,  0.0007,
         -0.0452,  0.0522, -0.0737,  0.0186,  0.0131,  0.0755,  0.0837,  0.0666,
          0.0034,  0.0283,  0.0242, -0.0115,  0.0652, -0.0060, -0.0175,  0.1019,
         -0.0611, -0.0398,  